# Apply few changes to WOA09 dataset before applying a vertical interpolation using SOSIE

In [15]:
# 0- Rename the vertical dimension and add a depth variable for the vertical interpolation if missing
# 1- Add missing value in file
# 2- Change the record time_counter because SOSIE need it (why ?) not done 
# 3- Add lat lon depth fields that have been removed by hand using the command line: 
#    ncks -v lat,lon,depth woa09_salinity_seasonal_1deg.nc woa09_salinity_seasonal_1deg_genvar.nc
# 4- Add a right axis time (extracted from the woa09_salinity_seasonal_1deg.nc) file using the command line:
#    ncks -v time woa09_salinity_monthly_1deg.nc woa09_monthly_time.nc 
#    rename -v time,time_counter woa09_monthly_time.nc  


In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm
import gsw as gsw
from netCDF4 import Dataset

from matplotlib import rcParams
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap

%matplotlib inline

In [2]:
! date

Thu Apr 17 15:54:25 CEST 2025


>>>> ## Get depth as time as well

In [21]:
INDIR='/data0/project/drakkar/REFERENCE_DATA/WOA2009/DATA/1degx1deg/COMBINED/'
ds_depth=xr.open_dataset(INDIR+'woa09_1deg_depth.nc')
#ds_latlonZ=ds_latlonZ.rename({'lat0':'lat','lon0':'lon'})
ds_depth

<xarray.Dataset>
Dimensions:  (depth: 33)
Coordinates:
  * depth    (depth) float32 0.0 10.0 20.0 30.0 ... 4e+03 4.5e+03 5e+03 5.5e+03
Data variables:
    *empty*
Attributes: (12/42)
    CVS_ID:                     1.0
    Conventions:                CF-1.4
    id:                         042eca21-20ac-4646-9c43-dfcc69308217
    naming_authority:           gov.noaa.nodc
    Metadata_Convention:        Unidata Dataset Discovery v1.0
    title:                      World Ocean Atlas 09:  Sea Water Salinity - s...
    ...                         ...
    time_coverage_resolution:   P3M
    processing_level:           synthesized product
    license:                    This data is free for anyone to use, reuse an...
    cdm_data_type:              Grid
    history:                    Thu Apr 17 13:32:50 2025: ncks -v depth woa09...
    NCO:                        "4.6.4"

In [6]:
INDIR='/data0/project/drakkar/REFERENCE_DATA/WOA2009/DATA/1degx1deg/COMBINED/'
ds_time=xr.open_dataset(INDIR+'woa09_monthly_time.nc',decode_times=False)
ds_time

<xarray.Dataset>
Dimensions:       (time_counter: 12)
Coordinates:
  * time_counter  (time_counter) float32 15.0 46.0 75.0 ... 289.0 320.0 350.0
Data variables:
    *empty*
Attributes: (12/42)
    CVS_ID:                     1.0
    Conventions:                CF-1.4
    id:                         592ac385-a915-4d55-8f34-0f85caf22756
    naming_authority:           gov.noaa.nodc
    Metadata_Convention:        Unidata Dataset Discovery v1.0
    title:                      World Ocean Atlas 09:  Sea Water Salinity - m...
    ...                         ...
    time_coverage_resolution:   P1M
    processing_level:           synthesized product
    license:                    This data is free for anyone to use, reuse an...
    cdm_data_type:              Grid
    history:                    Tue Jun 13 16:05:22 2017: ncrename -d time,ti...
    NCO:                        4.4.2

In [41]:
# Create tuples 
temp={'name':'temperature','namvar':'t_an'}
sal={'name':'salinity','namvar':'s_an'}

In [43]:
# Now loop on smooth level & variables
INDIR='/data0/project/drakkar/REFERENCE_DATA/WOA2009/DATA/1degx1deg/COMBINED/'
for zsm in np.arange(3)+1:
    for var in [temp,sal]:        
        print
        print(' Smoothing: '+str(zsm)+' variable : '+var['namvar'])
        ds_in=xr.open_dataset(INDIR+'woa09_'+var['name']+'_monthly_1deg_'+var['namvar']+'_CMA_SM'+str(zsm)+'.nc')
        ds_in
        ds_out=xr.Dataset()
        ds_out[var['namvar']]=(('time_counter','depth','lat','lon'),ds_in[var['namvar']].data)
        ds_out[var['namvar']].attrs['missing_value']=9.96921e+36

        ds_out.coords['lat']=(('lat'),ds_in['lat'].data)
        ds_out.coords['lon']=(('lon'),ds_in['lon'].data)
        ds_out.coords['depth']=(('depth'),ds_depth['depth'].data)
        ds_out.coords['time_counter']=(('time_counter'),ds_time['time_counter'].data)
        ds_out.attrs['History']=ds_in.attrs['History']+'\n'+ds_time.attrs['history']+'\n'+ds_depth.attrs['history']
        ds_out.attrs['summary']=ds_time.attrs['summary']
        ds_out.attrs['references']=ds_time.attrs['references']        
        ds_out.attrs['institution']=ds_time.attrs['institution']
        ds_out.attrs['project']=ds_time.attrs['project']        

        ds_out.to_netcdf(INDIR+'woa09_'+var['name']+'_monthly_1deg_'+var['namvar']+'_CMA_SM'+str(zsm)+'_4So.nc',unlimited_dims='time_counter')

 Smoothing: 2 variable : t_an
 Smoothing: 3 variable : t_an
